In [1]:
# Import libraries
import re
import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
import os, os.path as osp

import nltk  #to install: activate ada,   conda install -c anaconda nltk
#nltk.download('stopwords')
from nltk.tokenize import word_tokenize
#nltk.download('punkt')
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
import string


In [2]:
meat_products = ['steak','lamb', 'beef', 'cheese', 'pork', 'turkey', 'chicken', 'tuna', 'egg']


In [3]:
#Import CSV
DATA_FOLDER='data'
df=pd.read_csv(DATA_FOLDER+'/recipes_data.csv')
df
#recipes_df[recipes_df['Has meat']].count()
#74 out of the first 200 recipes have recognized formats.
#45 out of these 74 have "meat" ingredients


,Recipe Title,Has meat,Meat types,Meat quantity (kg),Carbon footprint,Rating
0,\r\n\tChicken Breast Cutlets with Artichokes a...,True,"['chicken', 'chicken']","[0.1511715797430083, 0.0]",1.043084,4.40
1,\r\n\tBest Ever Popcorn Balls Recipe - Allreci...,False,[],0,0.000000,4.40
2,Pumpkin Oatmeal Recipe : Aarti Sequeira : Reci...,False,[],0,0.000000,5.00
3,\r\n\tGreen Bean Casserole Recipe from Betty ...,False,[],0,0.000000,NaN
4,\r\n\tOrange Cream Cheese Frosting Recipe - Al...,True,['cheese'],[0.02834949254408346],0.382718,4.60
5,Orange Curd Recipe : Ina Garten : Recipes : Fo...,True,['egg'],[0.008],0.038400,4.00
6,Chocolate Chunk Cookies Recipe : Ina Garten : ...,True,['egg'],[0.00015789473684210527],0.000758,5.00
7,\r\n\tPerfect Baked Potato Recipe - Allrecipes...,True,['cheese'],[0.0],0.000000,4.70
8,\r\n\tPumpkin Oatmeal Recipe - Allrecipes.com\r\n,False,[],0,0.000000,3.90
9,\r\n\tBaked Asparagus with Balsamic Butter Sau...,False,[],0,0.000000,4.60


### Clustering 

In [5]:

#remove website and meat value from title (NEEDED FOR CLUSTERING)
def tolkenize_title(title_str, recipe_site):
    """
    
    input:
    title_str = full recipe title
    recipe_site = website where recipe is from
    output:
    word_tokens = only the relevant keywords from the title (excluding website, stopwords, punctuation and meat keywords)
    
    """
    #make title lowercase
    title_str=title_str.casefold()
    
    #remove recipe site
    title_str=re.sub(recipe_site,'',title_str)
    title_str=re.sub('.com', '',title_str)
    title_str=re.sub('recipe', '',title_str)
    if 'betty' in title_str: #betty crocker has the form 'bettycrocker' and 'betty crocker'
        title_str=re.sub('bettycrocker','', title_str)
        
    #remove meat products from title to avoid clustering meat with meat....MAYBE WE SHOULDN'T DO THIS..?
    for i in meat_products:
        title_str=re.sub(i, '',title_str)
    
    #remove punctuation
    title_str="".join((char for char in title_str if char not in string.punctuation))

    stop_words=set(stopwords.words('english'))
    word_tokens = word_tokenize(title_str)
    filtered_sentence = [w for w in word_tokens if not w in stop_words] 
    
    #print('word tokens=',word_tokens) 
    #print('filtered sentence = ',filtered_sentence) 
    #print('title str = ',title_str)

    return filtered_sentence

tolkenize_title('the Simple Collard Chicken of !! Greens Recipe | BettyCrocker.com', 'betty crocker')
    
    
    

['simple', 'collard', 'greens']

In [ ]:
#transfer corpus into vector space

#calculating distance between each title as a measure of similarity

#clustering the documents using the k-means algorithm


#using multidimensional scaling to reduce dimensionality within the corpus
